In [1]:
import pandas as pd
import numpy as np
import os
import time
from rdkit import Chem
import selfies as sf
from fp_gen import KlekFPGenerator, MACCSFPGenerator, SubFPGenerator

In [19]:
generator = KlekFPGenerator(n_jobs=16)
data = pd.read_csv('chembl_1m_smiles.csv', chunksize=5000, sep=';', header=0, names=['Smiles'])

alphabet = pd.read_csv('../GRU_data/alphabet.txt', header=None).values[:, 0]
alphabet

array(['[#Branch1]', '[#Branch2]', '[#C]', '[#N]', '[/C]', '[/N]', '[/O]',
       '[=Branch1]', '[=Branch2]', '[=C]', '[=N+1]', '[=N]', '[=O]',
       '[=Ring1]', '[=Ring2]', '[=S]', '[Br]', '[Branch1]', '[Branch2]',
       '[C@@H1]', '[C@@]', '[C@H1]', '[C@]', '[C]', '[Cl]', '[F]', '[I]',
       '[N+1]', '[NH1]', '[N]', '[O-1]', '[O]', '[P]', '[Ring1]',
       '[Ring2]', '[S]', '[\\\\C]', '[\\\\N]', '[\\\\O]', '[end]',
       '[nop]', '[start]'], dtype=object)

In [20]:
def could_be_valid(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False

In [21]:
def sparse_to_dense(sparse):
    return np.nonzero(sparse)[0].tolist()

In [ ]:
from tqdm import tqdm

for i, chunk in enumerate(tqdm(data)):
    print(i)
    chunk = chunk[chunk['Smiles'].apply(could_be_valid)]
    mols = list(map(Chem.MolFromSmiles, chunk['Smiles']))
    fps = generator.transform(mols)
    chunk['fps'] = list(map(sparse_to_dense, fps))
    if i == 0:
        chunk.to_csv('chembl_1m.csv', index=False, header=False)
    with open('chembl_1m.csv', 'a') as f:
        chunk.to_csv(f, header=False, index=False)

0it [00:00, ?it/s]

0


1it [00:27, 27.05s/it]

1


2it [00:56, 28.69s/it]

2


3it [01:27, 29.63s/it]

3


In [ ]:
reload_data = pd.read_csv('chembl_1m.csv')
reload_data['fps'] = reload_data['fps'].apply(eval)
reload_data.head()